In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
mta = pd.read_pickle("final_mta_v1.pkl")

In [4]:
mta

,STATION,DAILY_ENTRIES,NM_DAILY_ENTRIES
61,34 ST-PENN STA,11314018.0,1.000000
233,GRD CNTRL-42 ST,9793229.0,0.865583
59,34 ST-HERALD SQ,8354818.0,0.738447
46,23 ST,7881548.0,0.696617
68,42 ST-PORT AUTH,7092469.0,0.626873
...,...,...,...
124,ASTORIA DITMARS,1152389.0,0.101851
88,61 ST WOODSIDE,1148506.0,0.101508
23,167 ST,1146475.0,0.101329
358,UTICA AV,1130547.0,0.099921


In [5]:
subway = pd.read_csv('nyc_open_data_subway.csv', usecols = ["NAME"])
subway

,NAME
0,Astor Pl
1,Canal St
2,50th St
3,Bergen St
4,Pennsylvania Ave
...,...
468,Coney Island - Stillwell Av
469,34th St - Hudson Yards
470,72nd St
471,86th St


In [6]:
#create new field "STATION_CLEAN" to prepare strings for fuzzymerge
mta["STATION_CLEAN"] = mta["STATION"].str.title()
mta["STATION_CLEAN"] = mta["STATION_CLEAN"].str.replace("/", "-")
mta

,STATION,DAILY_ENTRIES,NM_DAILY_ENTRIES,STATION_CLEAN
61,34 ST-PENN STA,11314018.0,1.000000,34 St-Penn Sta
233,GRD CNTRL-42 ST,9793229.0,0.865583,Grd Cntrl-42 St
59,34 ST-HERALD SQ,8354818.0,0.738447,34 St-Herald Sq
46,23 ST,7881548.0,0.696617,23 St
68,42 ST-PORT AUTH,7092469.0,0.626873,42 St-Port Auth
...,...,...,...,...
124,ASTORIA DITMARS,1152389.0,0.101851,Astoria Ditmars
88,61 ST WOODSIDE,1148506.0,0.101508,61 St Woodside
23,167 ST,1146475.0,0.101329,167 St
358,UTICA AV,1130547.0,0.099921,Utica Av


In [7]:
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process

In [8]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].tolist()

    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m

    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2

    return df_1

In [9]:
#create function to add ordinal suffix to number
ordinal = lambda n: "%d%s" % (n,"tsnrhtdd"[(n//10%10!=1)*(n%10<4)*n%10::4])

In [10]:
#add ordinal suffix to street number (within string)
def insert_ordinal(string):
    a = []
    aa = []
    num_index = []
    for i in string:
        aa.append(i)
        if i.isdigit():
            a.append(i)
    for i in aa:
        if i.isdigit():
            num_index.append(aa.index(i))
    if a:
        c = ''.join([str(elem) for elem in a])
        cc = int(c)
        b = ordinal(cc)
        beg_ind = num_index[0]
        end_ind = num_index[-1]
        del aa[beg_ind:end_ind+1]
        aa.insert(beg_ind, b)
        bb = ''.join(aa)
        return bb
    else:
        return string

In [11]:
#Test function
insert_ordinal("57 AV")

'57th AV'

In [12]:
#Test function
insert_ordinal("St. George")

'St. George'

In [13]:
mta["STATION_CLEAN"] = mta["STATION_CLEAN"].apply(insert_ordinal)

In [14]:
fuzmerge = fuzzy_merge(mta,subway, "STATION_CLEAN", "NAME", threshold=80, limit=1)

In [15]:
#export fuzmerge dataframe to a csv
# manually match stations where fuzzymerge failed
fuzmerge.to_csv("fuzmerge.csv")

In [16]:
#import manual match
fuzmerge_final = pd.read_excel("fuzmerge.xlsx", usecols = ["STATION", "DAILY_ENTRIES", "NM_DAILY_ENTRIES", "Unnamed: 6"])

In [17]:
fuzmerge_final.rename(columns={"Unnamed: 6":"name"}, inplace = True)

In [18]:
fuzmerge_final

,STATION,DAILY_ENTRIES,NM_DAILY_ENTRIES,name
0,34 ST-PENN STA,11314018,1.000000,34th St - Penn Station
1,GRD CNTRL-42 ST,9793229,0.865583,Grand Central - 42nd St
2,34 ST-HERALD SQ,8354818,0.738447,Herald Sq - 34th St
3,23 ST,7881548,0.696617,23rd St
4,42 ST-PORT AUTH,7092469,0.626873,42nd St - Port Authority Bus Term
...,...,...,...,...
95,ASTORIA DITMARS,1152389,0.101851,Astoria - Ditmars Blvd
96,61 ST WOODSIDE,1148506,0.101508,Woodside - 61st St
97,167 ST,1146475,0.101329,167th St
98,UTICA AV,1130547,0.099921,Utica Ave


In [19]:
stations_w_data = pd.read_csv("https://raw.githubusercontent.com/dunleavyjason/project-1/Alexander/GeoSpatial%20Data/Stations_W_Data.csv", usecols = ["name", "geometry", "GEOID", "NAME_y", "Total Population - 25 years and over - Advanced Degrees", "Median Income - 25 years and over", "Total Female Population - 25 years and over"])

In [20]:
#drop duplicate stations - investigation indicated duplicates are due to multiple lines running through station
stations_w_data.drop_duplicates(subset = ["name"], inplace = True)

In [21]:
stations_w_data

,name,geometry,GEOID,NAME_y,Total Population - 25 years and over - Advanced Degrees,Median Income - 25 years and over,Total Female Population - 25 years and over
0,Astor Pl,POINT (-73.99106999861966 40.73005400028978),36061005700,"Census Tract 57, New York County, New York",2106,125268,1265
1,8th St - NYU,POINT (-73.99250799849149 40.73046499853991),36061005700,"Census Tract 57, New York County, New York",2106,125268,1265
2,Canal St,POINT (-74.00019299927328 40.71880300107709),36061004500,"Census Tract 45, New York County, New York",452,90167,347
5,Spring St,POINT (-73.99714100006673 40.72230099999366),36061004500,"Census Tract 45, New York County, New York",452,90167,347
6,50th St,POINT (-73.98384899986625 40.76172799961419),36061013100,"Census Tract 131, New York County, New York",1827,95203,1142
...,...,...,...,...,...,...,...
139,Bleecker St,POINT (-73.99465900006331 40.72591466682659),36061005502,"Census Tract 55.02, New York County, New York",1105,107250,703
141,8th Ave,POINT (-74.00257800104762 40.73977666638199),36061008300,"Census Tract 83, New York County, New York",1932,59348,1513
142,110th St,POINT (-73.94424999687163 40.79502000011311),36061017200,"Census Tract 172, New York County, New York",1122,27269,1835
146,5th Ave - Bryant Pk,POINT (-73.98196299856706 40.75382100064824),36061008400,"Census Tract 84, New York County, New York",1222,104282,753


In [22]:
final_merge_v2 = pd.merge(fuzmerge_final, stations_w_data, on='name', how='inner')
final_merge_v2.head(50)

,STATION,DAILY_ENTRIES,NM_DAILY_ENTRIES,name,geometry,GEOID,NAME_y,Total Population - 25 years and over - Advanced Degrees,Median Income - 25 years and over,Total Female Population - 25 years and over
0,34 ST-PENN STA,11314018,1.000000,34th St - Penn Station,POINT (-73.99105699913983 40.75037300003949),36061010100,"Census Tract 101, New York County, New York",799,89375,456
1,GRD CNTRL-42 ST,9793229,0.865583,Grand Central - 42nd St,POINT (-73.97671329925841 40.75180742981634),36061009200,"Census Tract 92, New York County, New York",1152,113281,703
2,34 ST-HERALD SQ,8354818,0.738447,Herald Sq - 34th St,POINT (-73.9879368338264 40.74964456009442),36061010100,"Census Tract 101, New York County, New York",799,89375,456
3,23 ST,7881548,0.696617,23rd St,POINT (-73.98659900207888 40.7398640004746),36061006800,"Census Tract 68, New York County, New York",4182,105560,2852
4,42 ST-PORT AUTH,7092469,0.626873,42nd St - Port Authority Bus Term,POINT (-73.98973500085859 40.7573079985515),36061011900,"Census Tract 119, New York County, New York",347,22131,423
5,14 ST-UNION SQ,7019937,0.620462,Union Sq - 14th St,POINT (-73.98995099881881 40.73467300099612),36061005200,"Census Tract 52, New York County, New York",1910,133245,1024
6,TIMES SQ-42 ST,6966121,0.615705,Times Sq - 42nd St,POINT (-73.98676800153976 40.75461199851542),36061011300,"Census Tract 113, New York County, New York",39,91250,21
7,FULTON ST,6260737,0.553359,Fulton St,POINT (-74.00762309323994 40.71016216530185),36061001501,"Census Tract 15.01, New York County, New York",3843,80574,3017
8,86 ST,5755149,0.508672,86th St,POINT (-73.96882849429672 40.78582304678557),36061014300,"Census Tract 143, New York County, New York",0,0,1
9,86 ST-2 AVE,1780128,0.157335,86th St,POINT (-73.96882849429672 40.78582304678557),36061014300,"Census Tract 143, New York County, New York",0,0,1


In [23]:
final_merge_v2.reindex()

,STATION,DAILY_ENTRIES,NM_DAILY_ENTRIES,name,geometry,GEOID,NAME_y,Total Population - 25 years and over - Advanced Degrees,Median Income - 25 years and over,Total Female Population - 25 years and over
0,34 ST-PENN STA,11314018,1.000000,34th St - Penn Station,POINT (-73.99105699913983 40.75037300003949),36061010100,"Census Tract 101, New York County, New York",799,89375,456
1,GRD CNTRL-42 ST,9793229,0.865583,Grand Central - 42nd St,POINT (-73.97671329925841 40.75180742981634),36061009200,"Census Tract 92, New York County, New York",1152,113281,703
2,34 ST-HERALD SQ,8354818,0.738447,Herald Sq - 34th St,POINT (-73.9879368338264 40.74964456009442),36061010100,"Census Tract 101, New York County, New York",799,89375,456
3,23 ST,7881548,0.696617,23rd St,POINT (-73.98659900207888 40.7398640004746),36061006800,"Census Tract 68, New York County, New York",4182,105560,2852
4,42 ST-PORT AUTH,7092469,0.626873,42nd St - Port Authority Bus Term,POINT (-73.98973500085859 40.7573079985515),36061011900,"Census Tract 119, New York County, New York",347,22131,423
...,...,...,...,...,...,...,...,...,...,...
56,1 AV,1231084,0.108807,1st Ave,POINT (-73.98168087489128 40.73097497580066),36061003400,"Census Tract 34, New York County, New York",3296,80142,2588
57,8 ST-NYU,1211944,0.107115,8th St - NYU,POINT (-73.99250799849149 40.73046499853991),36061005700,"Census Tract 57, New York County, New York",2106,125268,1265
58,ASTOR PL,1194662,0.105588,Astor Pl,POINT (-73.99106999861966 40.73005400028978),36061005700,"Census Tract 57, New York County, New York",2106,125268,1265
59,WHITEHALL S-FRY,1176768,0.104006,Whitehall St,POINT (-74.0130072374272 40.70314237359914),36061000900,"Census Tract 9, New York County, New York",1305,117578,673


In [24]:
final_merge_v2.to_csv("final_merge_v2.csv")